In [98]:
import tensorflow as tf

from tensorflow.keras.layers import (
    Dense,
    Flatten,
    Conv2D,
    BatchNormalization,
    MaxPooling2D,
    Dropout,
    GlobalAveragePooling2D,
)

from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist, cifar10
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
print(tf.__version__)
print(tf.config.list_physical_devices("GPU"))

2.17.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [209]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize the data
x_train = x_train.reshape(-1, 32, 32, 3).astype("float32")
x_test = x_test.reshape(-1, 32, 32, 3).astype("float32")


# Add a channel dimension

In [210]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3), (50000, 1), (10000, 1))

In [211]:
class CNNblock(tf.keras.layers.Layer):

    def __init__(self, filters, kernel_size=3):
        super(CNNblock, self).__init__()
        self.conv = Conv2D(
            filters,
            kernel_size,
            input_shape=(32, 32, 3),
            activation="relu",
            padding="same",
        )
        self.norm = BatchNormalization()

    def __call__(self, inputs, training=False):
        x = self.conv(inputs, training=training)
        x = self.norm(x, training=training)
        return x

In [212]:
class ResNet_Block(tf.keras.layers.Layer):
    def __init__(self, channels):
        super(ResNet_Block, self).__init__()
        self.cnn1 = CNNblock(channels[0])
        self.cnn2 = CNNblock(channels[1])
        self.cnn3 = CNNblock(channels[2])
        self.pool = MaxPooling2D()
        self.identity_mapping = Conv2D(channels[1], 1, padding="same")

    def call(self, input, training=False):
        x = self.cnn1(input, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input), training=training)
        x = self.pool(x)
        return x

In [213]:
class ResNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(ResNet, self).__init__()
        self.block1 = ResNet_Block([32, 32, 64])
        self.block2 = ResNet_Block([128, 128, 256])
        self.block3 = ResNet_Block([128, 256, 512])
        self.pool = GlobalAveragePooling2D()
        self.classifier = Dense(num_classes, activation="softmax")

    def call(self, input, training=False):
        x = self.block1(input, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x)
        x = self.classifier(x)
        return x

    def model(self):
        x = tf.keras.layers.Input(shape=(32, 32, 3))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [214]:
model = ResNet(10)

In [215]:
model.model().summary()

Model: "functional_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_17 (InputLayer)     │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net__block_51               │ (None, 16, 16, 64)     │        29,280 │
│ (ResNet_Block)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net__block_52               │ (None, 8, 8, 256)      │       526,976 │
│ (ResNet_Block)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ res_net__block_53               │ (None, 4, 4, 512)      │     1,839,744 │
│ (ResNet_Block)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_17     │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,401,130 (9.16 MB)

 Trainable params: 2,398,058 (9.15 MB)

 Non-trainable params: 3,072 (12.00 KB)

In [216]:
model.compile(
    optimizer=Adam(),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.1)

Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.4433 - loss: 1.6446 - val_accuracy: 0.3858 - val_loss: 2.1366
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6853 - loss: 0.8853 - val_accuracy: 0.6542 - val_loss: 1.0500
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.7805 - loss: 0.6295 - val_accuracy: 0.7596 - val_loss: 0.7100
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.8326 - loss: 0.4826 - val_accuracy: 0.7374 - val_loss: 0.8263
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.8801 - loss: 0.3481 - val_accuracy: 0.7632 - val_loss: 0.7572
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.9153 - loss: 0.2535 - val_accuracy: 0.7980 - val_loss: 0.6756
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.9353 - loss: 0.1867 - val_accuracy: 0.7588 - val_loss: 0.8158
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.9436 - loss: 0.1612 - 

In [217]:
model.evaluate(x_test, y_test, batch_size=64)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8003 - loss: 0.7827


[0.7720578908920288, 0.8023999929428101]